![terrainbento logo](../images/terrainbento_logo.png)


# Introduction to the terrainbento output writers.


## Overview
This tutorial shows example usage of the terrainbento boundary handlers. For comprehensive information about all options and defaults, refer to the [documentation](http://terrainbento.readthedocs.io/en/latest/). 

## Prerequisites
This tutorial assumes you have at least skimmed the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/) and worked through the [Introduction to terrainbento](http://localhost:8888/notebooks/example_usage/Introduction_to_terrainbento.ipynb) tutorial.


### terrainbento output writers

By default terrainbento will save a [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) file at equal intervals specified by the ``output_interval`` parameter. For many applications it may be sufficient to run a model, save output, and then open the stack of NetCDFs for postprocessing (we recommend [xarray](http://xarray.pydata.org/en/stable/) and [holoviews](http://holoviews.org) for opening, analyzing, and plotting stacks of NetCDFs in python). 

If in your application, you want to have additional output written at the ``output_interval`` intervals, you can pass any number of functions or classes that write this ouput to terrainbento. These functions and classes are called **output writers**. There is no limit to what you can do with the output writers. You could use them to make plots to turn into an animation, write diagnostic information about model run time, or calculate an objective function value over time. 

To make the output writers support as many possible use cases, terrainbento was designed to take any number of functions or classes as output writers. 

### Reference material
If you are not familiar with user defined python functions, consider reviewing [this tutorial](https://www.datacamp.com/community/tutorials/functions-python-tutorial#udf). If you are not familiar with user defined python classes, consider referring to [this tutorial](https://www.digitalocean.com/community/tutorials/how-to-construct-classes-and-define-objects-in-python-3).


### Example Usage

To begin, we will import the required python modules. 

In [ ]:
import numpy as np
np.random.seed(42)

import os
import glob

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from landlab.components import DrainageDensity
from landlab import imshow_grid

from terrainbento import Basic

We begin by creating an example user defined output writer function. When a terrainbento model runs the `OutputWriter` function, it will pass the model instance to the output writer. Thus the output writer function must be able to make all required calculations and plots based on the model instance. It is almost always the case that a user will want to write some sort of output -- typically a text file or plot -- each time the output writer is run. 

A user has the option of appending the output writer output to a file or using the model attributes `model.model_time` or `model.iteration` in the output file name to distinguish it from other files.  



In [ ]:
def name_of_output_writer_function(model):
    # put lines of code here that use the model object to calculate outputs
    # for example, mean elevation
    val = model.z.mean()

    # write output, option 1: unique files per timestep
    with open('name_of_file.' + str(int(model.iteration)) + '.txt', 'w') as f:
        f.write(str(val))

    # write output, option 2: append values to same file
    with open('name_of_file.txt', 'a') as f:
        f.write(
            str(model.iteration) + ', ' + str(model.model_time) + ', ' +
            str(val) + '\n')

Here we will define an output writer called `write_drainage_density`. Not surprisingly, it calculates and writes out the drainage density of a model domain. The **DrainageDensity** component has two options for identifying where channels are located -- either a user-specified channel mask or a slope-area threshold. We will use this latter option. 

In [ ]:
def write_drainage_density(model):
    dd = DrainageDensity(model.grid,
                         area_coefficient=0.1,
                         slope_coefficient=1.0,
                         area_exponent=0.5,
                         slope_exponent=0.6,
                         channelization_threshold=0.35)
    mean_drainage_density = dd.calc_drainage_density()
    if np.isinf(mean_drainage_density):
        mean_drainage_density = 0.0
    fname = 'drainage_density.txt'
    if os.path.exists(fname) is False:
        with open(fname, 'w') as f:
            f.write('model_time,drainage_density\n')

    with open(fname, 'a') as f:
        f.write(
            str(model.model_time) + ', ' + str(mean_drainage_density) + '\n')

    del dd

Next we construct the parameter dictionary we will use to run the model. 

In [ ]:
basic_params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 500,
        "stop": 2e5
    },

    # Create the Grid
    "grid": {
        "RasterModelGrid": [
            (50, 80),
            {
                "xy_spacing": 20
            },
            {
                "fields": {
                    "node": {
                        "topographic__elevation": {
                            "random": [{
                                "where": "CORE_NODE"
                            }]
                        }
                    }
                }
            },
        ]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "NotCoreNodeBaselevelHandler": {
            "modify_core_nodes": True,
            "lowering_rate": -0.0005
        }
    },
    # Parameters that control output.
    "output_interval": 1e3,
    "save_first_timestep": True,
    "output_prefix": "model_basic_output_intro_ow1",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": 0.001,
    "m_sp": 0.5,
    "n_sp": 1.0,
    "regolith_transport_parameter": 0.1,
}

Unline the boundary handlers described in [this tutorial](http://localhost:8888/notebooks/example_usage/introduction_to_boundary_conditions.ipynb), we pass the output writer to terrainbento as a keyword argument. 

After instantiating the model, we run it. 

In [ ]:
basic = Basic.from_dict(basic_params,
                        output_writers={"function": [write_drainage_density]})
basic.run()

First we will make a plot of the topography. We have evolved some nice valleys and ridges. 

In [ ]:
imshow_grid(basic.grid, basic.z)

Next we plot the channel mask from the last timestep. It is stored in an at-node grid field called `chanel__mask`.

In [ ]:
channel__mask = basic.grid.at_node['channel__mask'].astype(int)
imshow_grid(basic.grid, channel__mask)

Our mask identifies the channel areas. 

Next we will read in the output we wrote to the file `drainage_density.txt` using [pandas](https://pandas.pydata.org) and remove the files created by running the model. In your usage you probably won't want to remove these files, but we are doing it so that running the model won't create lots of files in your directory. 

In [ ]:
df = pd.read_csv('drainage_density.txt')
#basic.remove_output_netcdfs()
#os.remove('drainage_density.txt')

We can look at what is inside of the dataframe. 

In [ ]:
df.head()

In [ ]:
df.columns

This dataframe shows us the history of drainage density through time. Next we will plot it. 

In [ ]:
plt.figure()
plt.plot(df.model_time, df.drainage_density)
plt.xlabel('Model Time [years]')
plt.ylabel('Drainage Density')
plt.show()

### Using a class instead of a function

terrainbento can support output writer classes. It is expected that the class will look something like the following.

In [ ]:
class my_ouput_writer_class(object):
    def __init__(self, model):
        # save the model to the output writer class
        self.model = model
        # do whatever else you want to do here:
    def run_one_step(self):
        # make calculations

        # write something out based on the calculations.
        # we recommend including the model time as a part of
        # the filename so that it is possible to identify
        # what time each file is associated with

        pass  # remove this line when you modify this example.

You might ask: when is an output writer class more appropriate than a function? The answer is that if you can calculate everything you need from the model instance at a given timestep, you probably can just use a function. However, if you need information from past timesteps, you'll need to use a class. 

For example, here we will use a simple example in which the output writer writes out the mean and maximum elevation averaged over a few timesteps. We could not do this with a function. 

In [ ]:
class time_averaged_elevation(object):
    def __init__(self, model):
        self.model = model
        self.model_time = []
        self.model_mean_elevation = []
        self.model_max_elevation = []

    def run_one_step(self):
        self.model_time.append(self.model.model_time)
        self.model_mean_elevation.append(
            self.model.z[self.model.grid.core_nodes].mean())
        self.model_max_elevation.append(
            self.model.z[self.model.grid.core_nodes].max())
        if len(self.model_time) >= 5:
            with open(
                    'time_averaged_elevation.' +
                    str(int(self.model.model_time)) + '.txt', 'w') as f:
                f.write(str(np.mean(self.model_mean_elevation[-5:])) + '\n')
                f.write(str(np.mean(self.model_max_elevation[-5:])) + '\n')

Next we run the model with the new output writer. Note that if you want to use more than one output writer you can just provide a list of functions and classes. 

In [ ]:
basic_params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 500,
        "stop": 2e5
    },

    # Create the Grid
    "grid": {
        "RasterModelGrid": [
            (50, 80),
            {
                "xy_spacing": 20
            },
            {
                "fields": {
                    "node": {
                        "topographic__elevation": {
                            "random": [{
                                "where": "CORE_NODE"
                            }]
                        }
                    }
                }
            },
        ]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "NotCoreNodeBaselevelHandler": {
            "modify_core_nodes": True,
            "lowering_rate": -0.0005
        }
    },
    # Parameters that control output.
    "output_interval": 1e3,
    "save_first_timestep": True,
    "output_prefix": "model_basic_output_intro_ow2",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": 0.001,
    "m_sp": 0.5,
    "n_sp": 1.0,
    "regolith_transport_parameter": 0.1,
}

basic = Basic.from_dict(basic_params,
                        output_writers={
                            "function": [write_drainage_density],
                            "class": [time_averaged_elevation]
                        })

basic.run()

Now that we've run the model we will compile all of the output into a dataframe and plot it. 

In [ ]:
output_files = glob.glob('time_averaged_elevation.*.txt')
temp = {}
for of in output_files:
    with open(of, 'r') as f:
        model_time = int(of.split('.')[-2])
        mean_elev = float(f.readline())
        max_elev = float(f.readline())
        temp[model_time] = {
            'mean_elevation': mean_elev,
            "max_elevation": max_elev
        }
    os.remove(of)
df = pd.DataFrame(temp).T
df = df.sort_index()

basic.remove_output_netcdfs()
os.remove('drainage_density.txt')

plt.figure()
plt.plot(df.index, df.mean_elevation, label='Mean')
plt.plot(df.index, df.max_elevation, label='Max')
plt.legend()
plt.xlabel('Model Time [years]')
plt.ylabel('Elevation [meters]')
plt.show()

In [ ]:
df.head()

## Next Steps

- We recommend you review the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/).

**The links to other notebooks will not work on GitHub. To use them you'll need to clone this repository and open the notebooks using [`jupyter notebook`](https://jupyter.org).**

- There are three additional introductory tutorials: 

    1) [Introduction terrainbento](http://localhost:8888/notebooks/notebooks/example_usage/Introduction_to_terrainbento.ipynb) 
    
    2) [Introduction to boundary conditions in terrainbento](http://localhost:8888/notebooks/notebooks/example_usage/introduction_to_boundary_conditions.ipynb)
    
    3) **This Notebook**: [Introduction to output writers in terrainbento](http://localhost:8888/notebooks/notebooks/example_usage/introduction_to_output_writers.ipynb). 
    
    
- Five examples of steady state behavior in coupled process models can be found in the following notebooks:

    1) [Basic](http://localhost:8888/notebooks/notebooks/coupled_process_elements/model_basic_steady_solution.ipynb) the simplest landscape evolution model in the terrainbento package.

    2) [BasicVm](http://localhost:8888/notebooks/notebooks/coupled_process_elements/model_basic_var_m_steady_solution.ipynb) which permits the drainage area exponent to change

    3) [BasicCh](http://localhost:8888/notebooks/notebooks/coupled_process_elements/model_basicCh_steady_solution.ipynb) which uses a non-linear hillslope erosion and transport law

    4) [BasicVs](http://localhost:8888/notebooks/notebooks/coupled_process_elements/model_basicVs_steady_solution.ipynb) which uses variable source area hydrology

    5) [BasisRt](http://localhost:8888/notebooks/notebooks/coupled_process_elements/model_basicRt_steady_solution.ipynb) which allows for two lithologies with different K values